In [2]:
import os
import time
import datetime
import requests
import pandas as pd
import pyarrow
import numpy as np
from prefect import flow, task # Prefect flow and task decorators
import time
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

In [5]:
#read firms_output parquet
df_thai = pd.read_parquet("df_thai", engine="pyarrow")
df_thai.info()
df_thai.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7240 entries, 0 to 7239
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   country_id  7240 non-null   object  
 1   latitude    7240 non-null   float64 
 2   longitude   7240 non-null   float64 
 3   brightness  7240 non-null   float64 
 4   scan        7240 non-null   float64 
 5   track       7240 non-null   float64 
 6   acq_date    7240 non-null   object  
 7   acq_time    7240 non-null   object  
 8   satellite   7240 non-null   object  
 9   instrument  7240 non-null   object  
 10  confidence  7240 non-null   int64   
 11  version     7240 non-null   object  
 12  bright_t31  7240 non-null   float64 
 13  frp         7240 non-null   float64 
 14  daynight    7240 non-null   object  
 15  acq_year    7240 non-null   category
 16  acq_month   7240 non-null   category
 17  acq_day     7240 non-null   category
 18  acq_hour    7240 non-null   category
 19  acq_mi

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,13.09594,101.30117,304.84,1.75,1.30,2025-03-18,14:14:00,Terra,MODIS,56,6.1NRT,294.67,9.10,N,2025,3,18,14,14
1,THA,13.32354,101.35509,306.18,1.72,1.29,2025-03-18,14:14:00,Terra,MODIS,62,6.1NRT,294.31,10.12,N,2025,3,18,14,14
2,THA,13.35216,102.19923,304.28,1.49,1.21,2025-03-18,14:14:00,Terra,MODIS,52,6.1NRT,294.26,6.78,N,2025,3,18,14,14
3,THA,13.43091,102.04161,305.03,1.53,1.22,2025-03-18,14:14:00,Terra,MODIS,56,6.1NRT,294.42,7.93,N,2025,3,18,14,14
4,THA,13.43438,102.03480,306.52,1.53,1.22,2025-03-18,14:14:00,Terra,MODIS,63,6.1NRT,294.32,9.21,N,2025,3,18,14,14


In [4]:
duplicate = df_thai[df_thai.duplicated()]
 
print("Duplicate Rows :")
 
# Print the resultant Dataframe
duplicate

Duplicate Rows :


,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,acq_year,acq_month,acq_day,acq_hour,acq_minute
849,THA,13.47638,102.12600,308.84,1.39,1.17,2025-03-23,14:16:00,Terra,MODIS,73,6.1NRT,294.79,11.52,N,2025,3,23,14,16
850,THA,13.51879,101.98978,312.56,1.42,1.18,2025-03-23,14:16:00,Terra,MODIS,84,6.1NRT,295.65,15.33,N,2025,3,23,14,16
851,THA,13.52072,102.00252,310.62,1.42,1.18,2025-03-23,14:16:00,Terra,MODIS,78,6.1NRT,295.83,13.11,N,2025,3,23,14,16
852,THA,13.52707,101.99515,328.84,1.42,1.18,2025-03-23,14:16:00,Terra,MODIS,100,6.1NRT,296.35,42.60,N,2025,3,23,14,16
853,THA,13.56741,102.11742,306.44,1.39,1.17,2025-03-23,14:16:00,Terra,MODIS,63,6.1NRT,295.52,8.62,N,2025,3,23,14,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7115,THA,13.12941,101.16443,313.36,2.51,1.52,2025-04-24,03:17:00,Terra,MODIS,22,6.1NRT,293.97,21.80,D,2025,4,24,3,17
7116,THA,14.23789,99.99210,317.42,1.93,1.36,2025-04-24,03:17:00,Terra,MODIS,67,6.1NRT,297.77,14.23,D,2025,4,24,3,17
7117,THA,14.24072,99.97489,318.01,1.93,1.36,2025-04-24,03:17:00,Terra,MODIS,68,6.1NRT,297.64,15.57,D,2025,4,24,3,17
7118,THA,14.40648,100.01643,319.74,1.93,1.36,2025-04-24,03:17:00,Terra,MODIS,71,6.1NRT,299.24,16.49,D,2025,4,24,3,17


In [4]:
import datetime

# Ensure "acq_time" is a string, then pad with zeros
df_thai["acq_time"] = df_thai["acq_time"].astype(str).str.zfill(4)

# Convert to HH:MM format
df_thai["acq_time"] = pd.to_datetime(df_thai["acq_time"], format="%H%M").dt.time
# df_thai["acq_time"] = df_thai["acq_time"].apply(lambda x: datetime.datetime.strptime(x, "%H:%M:%S").time())

# Check the result
print(df_thai.dtypes)
print(df_thai["acq_time"].head())  # Display first few converted times

ValueError: unconverted data remains when parsing with format "%H%M": ":14:00", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [5]:
#create coloumn acq_datetime
df_thai['acq_datetime'] = pd.to_datetime(df_thai['acq_date'].astype(str) + ' ' + df_thai['acq_time'].astype(str), format='%Y-%m-%d %H:%M:%S')
df_thai['acq_datetime'].head(5)

0   2025-03-18 14:14:00
1   2025-03-18 14:14:00
2   2025-03-18 14:14:00
3   2025-03-18 14:14:00
4   2025-03-18 14:14:00
Name: acq_datetime, dtype: datetime64[ns]

In [6]:
import pytz
#change timazone to Bangkok
df_thai['acq_datetime_th'] = df_thai['acq_datetime'].dt.tz_localize('GMT').dt.tz_convert('Asia/Bangkok')
df_thai['acq_datetime_th'].head(5)

0   2025-03-18 21:14:00+07:00
1   2025-03-18 21:14:00+07:00
2   2025-03-18 21:14:00+07:00
3   2025-03-18 21:14:00+07:00
4   2025-03-18 21:14:00+07:00
Name: acq_datetime_th, dtype: datetime64[ns, Asia/Bangkok]

In [7]:
unique_values = df_thai["acq_time"].unique()
print(unique_values)

['14:14:00' '14:16:00' '20:02:00' '20:04:00' '03:25:00' '03:28:00'
 '07:52:00' '07:54:00' '14:54:00' '14:57:00' '02:26:00' '06:54:00'
 '08:33:00' '13:57:00' '19:42:00' '19:45:00' '03:06:00' '07:31:00'
 '07:33:00' '14:35:00' '14:37:00' '20:21:00' '08:11:00' '15:16:00'
 '02:47:00' '02:49:00' '07:12:00' '07:14:00' '14:18:00' '07:48:00'
 '07:50:00' '14:59:00' '02:28:00' '20:40:00' '06:52:00' '08:30:00'
 '13:59:00' '03:10:00' '03:08:00' '07:29:00' '14:40:00' '02:09:00'
 '20:19:00' '08:09:00' '15:18:00' '19:21:00' '02:51:00' '07:09:00'
 '14:20:00' '20:00:00' '03:30:00' '02:30:00' '08:28:00' '14:01:00'
 '14:42:00' '13:42:00' '15:20:00' '02:53:00' '14:44:00' '20:17:00'
 '14:25:00' '19:57:00' '03:32:00' '07:44:00' '07:46:00' '15:03:00'
 '02:34:00' '06:48:00' '14:03:00' '19:38:00' '03:15:00' '07:24:00'
 '07:26:00' '02:56:00' '07:07:00' '08:24:00' '14:05:00' '14:22:00'
 '08:05:00' '08:07:00' '02:58:00' '07:05:00' '14:27:00' '19:55:00'
 '15:05:00' '15:07:00' '02:36:00' '14:08:00' '03:17:00' '02:17

In [9]:
df_thai.info()
df_thai.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7240 entries, 0 to 7239
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype                       
---  ------           --------------  -----                       
 0   country_id       7240 non-null   object                      
 1   latitude         7240 non-null   float64                     
 2   longitude        7240 non-null   float64                     
 3   brightness       7240 non-null   float64                     
 4   scan             7240 non-null   float64                     
 5   track            7240 non-null   float64                     
 6   acq_date         7240 non-null   object                      
 7   acq_time         7240 non-null   object                      
 8   satellite        7240 non-null   object                      
 9   instrument       7240 non-null   object                      
 10  confidence       7240 non-null   int64                       
 11  version          

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_year,acq_month,acq_day,acq_hour,acq_minute,acq_datetime,acq_datetime_th
0,THA,13.09594,101.30117,304.84,1.75,1.30,2025-03-18,14:14:00,Terra,MODIS,...,294.67,9.10,N,2025,3,18,14,14,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00
1,THA,13.32354,101.35509,306.18,1.72,1.29,2025-03-18,14:14:00,Terra,MODIS,...,294.31,10.12,N,2025,3,18,14,14,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00
2,THA,13.35216,102.19923,304.28,1.49,1.21,2025-03-18,14:14:00,Terra,MODIS,...,294.26,6.78,N,2025,3,18,14,14,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00
3,THA,13.43091,102.04161,305.03,1.53,1.22,2025-03-18,14:14:00,Terra,MODIS,...,294.42,7.93,N,2025,3,18,14,14,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00
4,THA,13.43438,102.03480,306.52,1.53,1.22,2025-03-18,14:14:00,Terra,MODIS,...,294.32,9.21,N,2025,3,18,14,14,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,THA,15.07057,100.86523,325.33,1.20,1.09,2025-03-18,07:52:00,Aqua,MODIS,...,306.82,8.91,D,2025,3,18,7,52,2025-03-18 07:52:00,2025-03-18 14:52:00+07:00
96,THA,15.19680,98.46285,323.08,1.00,1.00,2025-03-18,07:52:00,Aqua,MODIS,...,305.05,9.77,D,2025,3,18,7,52,2025-03-18 07:52:00,2025-03-18 14:52:00+07:00
97,THA,15.22165,99.61573,333.79,1.06,1.03,2025-03-18,07:52:00,Aqua,MODIS,...,298.37,25.21,D,2025,3,18,7,52,2025-03-18 07:52:00,2025-03-18 14:52:00+07:00
98,THA,15.25882,102.30477,320.12,1.49,1.20,2025-03-18,07:52:00,Aqua,MODIS,...,296.71,13.24,D,2025,3,18,7,52,2025-03-18 07:52:00,2025-03-18 14:52:00+07:00


Reorder columns

In [11]:
cols = list(df_thai.columns)
print(cols)

['country_id', 'latitude', 'longitude', 'brightness', 'scan', 'track', 'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence', 'version', 'bright_t31', 'frp', 'daynight', 'acq_year', 'acq_month', 'acq_day', 'acq_hour', 'acq_minute', 'acq_datetime', 'acq_datetime_th']


In [12]:
df_thai = df_thai[['country_id', 'latitude', 'longitude', 'brightness', 'scan', 'track', 'acq_date', 'acq_time', 'satellite',
                    'instrument', 'confidence', 'version','bright_t31', 'frp', 'daynight','acq_datetime', 'acq_datetime_th',
                      'acq_year', 'acq_month', 'acq_day', 'acq_hour', 'acq_minute']]


In [13]:
df_thai.info()
df_thai.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7240 entries, 0 to 7239
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype                       
---  ------           --------------  -----                       
 0   country_id       7240 non-null   object                      
 1   latitude         7240 non-null   float64                     
 2   longitude        7240 non-null   float64                     
 3   brightness       7240 non-null   float64                     
 4   scan             7240 non-null   float64                     
 5   track            7240 non-null   float64                     
 6   acq_date         7240 non-null   object                      
 7   acq_time         7240 non-null   object                      
 8   satellite        7240 non-null   object                      
 9   instrument       7240 non-null   object                      
 10  confidence       7240 non-null   int64                       
 11  version          

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,13.09594,101.30117,304.84,1.75,1.30,2025-03-18,14:14:00,Terra,MODIS,...,294.67,9.10,N,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00,2025,3,18,14,14
1,THA,13.32354,101.35509,306.18,1.72,1.29,2025-03-18,14:14:00,Terra,MODIS,...,294.31,10.12,N,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00,2025,3,18,14,14
2,THA,13.35216,102.19923,304.28,1.49,1.21,2025-03-18,14:14:00,Terra,MODIS,...,294.26,6.78,N,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00,2025,3,18,14,14
3,THA,13.43091,102.04161,305.03,1.53,1.22,2025-03-18,14:14:00,Terra,MODIS,...,294.42,7.93,N,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00,2025,3,18,14,14
4,THA,13.43438,102.03480,306.52,1.53,1.22,2025-03-18,14:14:00,Terra,MODIS,...,294.32,9.21,N,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00,2025,3,18,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,THA,15.07057,100.86523,325.33,1.20,1.09,2025-03-18,07:52:00,Aqua,MODIS,...,306.82,8.91,D,2025-03-18 07:52:00,2025-03-18 14:52:00+07:00,2025,3,18,7,52
96,THA,15.19680,98.46285,323.08,1.00,1.00,2025-03-18,07:52:00,Aqua,MODIS,...,305.05,9.77,D,2025-03-18 07:52:00,2025-03-18 14:52:00+07:00,2025,3,18,7,52
97,THA,15.22165,99.61573,333.79,1.06,1.03,2025-03-18,07:52:00,Aqua,MODIS,...,298.37,25.21,D,2025-03-18 07:52:00,2025-03-18 14:52:00+07:00,2025,3,18,7,52
98,THA,15.25882,102.30477,320.12,1.49,1.20,2025-03-18,07:52:00,Aqua,MODIS,...,296.71,13.24,D,2025-03-18 07:52:00,2025-03-18 14:52:00+07:00,2025,3,18,7,52


In [14]:
import numpy as np
import pandas as pd
import pyarrow

In [15]:
# Extract components from the "acq_date" and "acq_time" columns
df_thai["acq_year"] = pd.to_datetime(df_thai["acq_datetime_th"]).dt.year
df_thai["acq_month"] = pd.to_datetime(df_thai["acq_datetime_th"]).dt.month
df_thai["acq_day"] = pd.to_datetime(df_thai["acq_datetime_th"]).dt.day
df_thai["acq_hour"] = pd.to_datetime(df_thai["acq_datetime_th"]).dt.hour
df_thai["acq_minute"] = pd.to_datetime(df_thai["acq_datetime_th"]).dt.minute

In [16]:
df_thai.head(5)

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,...,bright_t31,frp,daynight,acq_datetime,acq_datetime_th,acq_year,acq_month,acq_day,acq_hour,acq_minute
0,THA,13.09594,101.30117,304.84,1.75,1.30,2025-03-18,14:14:00,Terra,MODIS,...,294.67,9.10,N,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00,2025,3,18,21,14
1,THA,13.32354,101.35509,306.18,1.72,1.29,2025-03-18,14:14:00,Terra,MODIS,...,294.31,10.12,N,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00,2025,3,18,21,14
2,THA,13.35216,102.19923,304.28,1.49,1.21,2025-03-18,14:14:00,Terra,MODIS,...,294.26,6.78,N,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00,2025,3,18,21,14
3,THA,13.43091,102.04161,305.03,1.53,1.22,2025-03-18,14:14:00,Terra,MODIS,...,294.42,7.93,N,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00,2025,3,18,21,14
4,THA,13.43438,102.03480,306.52,1.53,1.22,2025-03-18,14:14:00,Terra,MODIS,...,294.32,9.21,N,2025-03-18 14:14:00,2025-03-18 21:14:00+07:00,2025,3,18,21,14


In [17]:
df_thai.to_parquet(
    "firms_outout",
    partition_cols=["acq_year","acq_month","acq_day","acq_hour","acq_minute"],   # <-- crucial for partitioning by retrieval_time
    engine="pyarrow",
    index=False
)

In [18]:
df_read = pd.read_parquet("firms_outout", engine="pyarrow")
df_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7557 entries, 0 to 7556
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype                       
---  ------           --------------  -----                       
 0   country_id       7557 non-null   object                      
 1   latitude         7557 non-null   float64                     
 2   longitude        7557 non-null   float64                     
 3   brightness       7557 non-null   float64                     
 4   scan             7557 non-null   float64                     
 5   track            7557 non-null   float64                     
 6   acq_date         7557 non-null   object                      
 7   acq_time         7557 non-null   object                      
 8   satellite        7557 non-null   object                      
 9   instrument       7557 non-null   object                      
 10  confidence       7557 non-null   int64                       
 11  version          